<a href="https://colab.research.google.com/github/hungpham13/Bug-localization/blob/main/BL_Feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bug localization

In [ ]:
!git clone https://github.com/weedharm/AISE-dataset.git

Cloning into 'AISE-dataset'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 161 (delta 19), reused 150 (delta 14), pack-reused 0
Receiving objects: 100% (161/161), 286.37 MiB | 20.56 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Checking out files: 100% (123/123), done.


In [ ]:
import os
!pip install javalang
import javalang
import pathlib as pl
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import traceback

Unzip source code

In [ ]:
src_dir = "/content/AISE-dataset/Bug Localization Data/source files/"
for n in os.listdir(src_dir):
    !unzip -q "{src_dir+n}" -d ./unzip_source

Get all source file from project

In [ ]:
def get_all_source(root):
    all_source_path = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            #just source file
            if name[-5:] == '.java':
                all_source_path.append(str(pl.PurePath(path, name)))
    return sorted(all_source_path)

# src_dir =  "/content/unzip_source/"+ next(iter(sorted(os.listdir('./unzip_source'))))
src_dir =  "/content/unzip_source/birt-20140211-1400/"

def to_relative_path(full_path, src_dir=src_dir):
    assert full_path[:len(src_dir)] == src_dir
    return full_path[len(src_dir):]
def to_full_path(relative_path, src_dir=src_dir):
    return src_dir + relative_path

all_src_path = get_all_source(src_dir)

In [ ]:
def print_long_list(tokens, limit = 'all'):
    if limit == 'all':
        start = 0
        limit = len(tokens)
    else:
        start, limit = limit
    for i in range(start, limit, 10):
        for k in range(i, i+10):
            if k == limit:
                break
            print(tokens[k],end="\t")
            # print("\t")
        print("\n")

def get_content(file_path):
    with open(file_path, 'rb') as f:
        text = f.read().decode(errors='replace')
        return text.replace('\ufffd',"")

def get_ast(file_path, print_result = False):
    with open(file_path) as f:
        tree = javalang.parse.parse(f.read())
    if print_result:
        for path, node in tree:
            # print(path)
            print(node)
    return tree

def get_token(string, print_result = False, ignore_error= True):
    tokens = javalang.tokenizer.tokenize(string, ignore_errors=ignore_error)
    tokens = [t.value for t in tokens]

    if print_result:
        print_long_list(tokens)

    return tokens

## Source code cleaning
Abandon Number?
like `name.substring( index + 1 );`

or 

`NumberDataSetImpl.create( new double[]{143.26, 47.56, 35.8, 0, 95.25, 0, 123.45} );`


In [ ]:
#wiki/java_syntax

stop_word = """abstract	continue	for	new	switch
assert default	goto	package	synchronized
boolean	do	if	private	this 
break	double	implements	protected	throw
byte	else	import	public	throws
case	enum instanceof	return	transient
catch	extends	int	short	try
char	final	interface	static	void
class	finally	long	strictfp	volatile
const	float	native	super	while"""
black_list = [i+"[\s.:;\n\t]" for i in stop_word.split()]
black_list.append('\/\/.+\n') # comment // in java
black_list.append('0x[\w\d]+') # hex 0x00000
black_list.append('<[\w/!\-.]+>') # html tag <\>
black_list.append('\".+\"') # string "..."
black_list.append('["#(\[{\s.:;,\n\t]-?[\dabcdef]+["dDfFLleE\s.,:;\n\t})\]]|[\s.,:;\n\t]\d+$|^\d+[\s.,:;\n\t]') # number
black_list.append(';[\d;]+') # number sequence

punc = """
( )	 [ ]	 ++ -- + - ! ~ * / % << >> >>>	 < <= > >= instanceof	 == !=	 & ^ |	 &&
|| ? : =	 += -=	 *= /= %=	 <<= >>= >>>=	 &= ^= |= ; { }  ,  . " \\ # ` @
"""
punc_list = punc.split()

def clean_source(string, black_list = black_list, punc_list = punc_list):
    for k in black_list:
        string = re.sub(k, ";", string)
    for p in punc_list:
        string = string.replace(p, " ")
    string = string.replace("'", "")
    return string

In [ ]:
for i,f in enumerate(all_src_path):
    if "142948and141927" in get_content(f):
        print(f)
        print(i)
        break
# i = 7968
content = get_content(all_src_path[i])
print(content)
# print("-----------------------Cleaned------------------------")
print(clean_source(content))
# print("----------------------Tokenize------------------------")
# print(get_token(clean_source(content),print_result = True))

/content/unzip_source/birt-20140211-1400/testsuites/org.eclipse.birt.report.tests.model/src/org/eclipse/birt/report/tests/model/regression/AllRegressionTests.java
8775
package org.eclipse.birt.report.tests.model.regression;


import junit.framework.Test;
import junit.framework.TestSuite;

/**
 * Tests cases in activity package.
 */

public class AllRegressionTests
{

	/**
	 * @return the test
	 */

	public static Test suite( )
	{
		TestSuite test = new TestSuite( );

		// add all test classes here
		test.addTestSuite( Regression_100759.class );
		test.addTestSuite( Regression_101832.class );
		test.addTestSuite( Regression_102003.class );
		test.addTestSuite( Regression_102725.class );
		test.addTestSuite( Regression_102808.class );
		test.addTestSuite( Regression_112910.class );
		test.addTestSuite( Regression_116396.class );
		test.addTestSuite( Regression_116425.class );
		test.addTestSuite( Regression_116558.class );
		test.addTestSuite( Regression_116563.class );
		test.addTestSui

## Bug report cleaning

Import bug report to dataframe

In [ ]:
bugrep_file = "/content/AISE-dataset/Bug Localization Data/bug reports/Birt.xlsx"
bug_rp_df = pd.read_excel(bugrep_file)
bug_rp_df

,id,bug_id,summary,description,report_time,report_timestamp,status,commit,commit_timestamp,files,Unnamed: 10
0,1,422205,Bug 422205 Missing help content reference in M...,The help context ID is not referenced by the M...,2013-11-20 20:05:32,1385000000,resolved fixed,345f01b,1385000000,data/org.eclipse.birt.data.oda.mongodb.ui/src/...,2354:data/org.eclipse.birt.data.oda.mongodb.ui...
1,2,389229,Bug 389229 Documentation of possible values/ob...,NaN,2012-09-11 05:16:02,1347360000,resolved fixed,37c1c08,1382650000,chart/org.eclipse.birt.chart.engine/src/org/ec...,6:chart/org.eclipse.birt.chart.engine/src/org/...
2,3,403306,Bug 403306 Nested DataEngineSessions and wrong...,I have following use case: - Data Cube with me...,2013-03-14 05:52:07,1363250000,closed fixed,8299b4a,1381480000,data/org.eclipse.birt.data/src/org/eclipse/bir...,1:data/org.eclipse.birt.data/src/org/eclipse/b...
3,4,375294,Bug 375294 JBoss + Sybase database issue,"Build Identifier: 3.7.100.v20110510-0712 Hi, J...",2012-03-26 05:12:15,1332750000,resolved fixed,ffe1c1f,1367050000,data/org.eclipse.birt.report.data.oda.jdbc/src...,10:data/org.eclipse.birt.report.data.oda.jdbc/...
4,5,416987,Bug 416987 ArrayIndexOutOfBoundsException when...,There is a bug in org.eclipse.birt.report.engi...,2013-09-11 04:59:50,1378890000,resolved fixed,5eeefcf,1380240000,engine/org.eclipse.birt.report.engine/src/org/...,437:engine/org.eclipse.birt.report.engine/src/...
...,...,...,...,...,...,...,...,...,...,...,...
4173,4174,101361,Bug 101361 Bugs in org.eclipse.birt.report.eng...,It appears that there are at least two array i...,2005-06-22 17:07:33,1119470000,resolved fixed,d42a645,1120020000,engine/org.eclipse.birt.report.engine/src/org/...,NaN
4174,4175,101821,Bug 101821 Each time you edit one property in ...,Description: Each time you edit one property i...,2005-06-27 02:58:40,1119860000,closed fixed,8603039,1120020000,UI/org.eclipse.birt.report.designer.ui/src/org...,NaN
4175,4176,101838,Bug 101838 Maximum number of rows in data set ...,Description: Maximum number of rows in data se...,2005-06-27 04:58:21,1119860000,closed fixed,8259a65,1120010000,data/org.eclipse.birt.data/src/org/eclipse/bir...,NaN
4176,4177,101827,Bug 101827 Exception should be processed befor...,descrption: steps to reproduce: 1.go to chart ...,2005-06-27 03:58:32,1119860000,closed fixed,c263568,1120010000,chart/org.eclipse.birt.chart.engine/src/org/ec...,NaN


In [ ]:
def get_content_bug_rp(df):
    """input: dataframe
    return: a series that contains summary concat with description of each bug report"""
    cleaned_summary = df['summary'].str.extract(r'Bug[ \d]+(.+)',expand = False)
    s = cleaned_summary + ". " + df['description'].fillna("")
    return s.str.lower().tolist()

#test
i = 69
cleaned_summary = bug_rp_df['summary'].str.extract(r'Bug[ \d]+(.+)',expand = False)
print(cleaned_summary.iloc[i])
print(bug_rp_df['description'].iloc[i])
s = cleaned_summary + ". " + bug_rp_df['description'].fillna("")
s.str.lower().tolist()[i]

Refactor data set editor pages' codes[26441]
Build Identifier: 20100528 Refactor data set editor pages' codes. Reproducible: Always


"refactor data set editor pages' codes[26441]. build identifier: 20100528 refactor data set editor pages' codes. reproducible: always"

## Feature 1: TF-IDF


Haven't done yet: *Compound words such as “WorkBench” are split into their components based on capital letters*

In [ ]:
def vocab_construct(src_files = all_src_path):
    "Construct vocab from all source files"
    tokens = []
    for f in src_files:
        token = np.unique(get_token(clean_source(get_content(f))))
        tokens.extend(token)
    tokens = np.unique(tokens)
    #clean token, remove all cmt
    tokens = clean_source('  '.join(tokens)).split()
    # print(tokens)

    # enrich vocab by split into their components based on capital letters
    new = []
    # for t in tokens:
    #     new.extend(re.sub( r"([A-Z\d]*[a-z\d]*)", r" \1", t).split())
    return np.unique([i.lower() for i in np.append(tokens, new)])

def tf_idf(vocab, docs):
    """matrix of weight of each terms in vocab to each document"""
    tidf = TfidfVectorizer(vocabulary = vocab)
    X = tidf.fit_transform(docs)
    del tidf
    return X.toarray()

In [ ]:
vocab = vocab_construct()
# no_number_vocab = np.unique(clean_source('  '.join(vocab)).split())

### Porter stemmer
Finally, all words are reduced to
their stem using the Porter stemmer, as implemented in the
NLTK1 package. This process will reduce derivationally related words such as “programming” and “programs” to the
same stem “program”, which is known to have a positive
impact on the recall performance of the final system

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
# plurals = ['caresses', 'flies', 'dies', 'mules', 'denied', 
#            'died', 'agreed', 'owned', 'humbled', 'sized', 
#            'meeting', 'stating', 'siezing', 'itemization', 
#            'sensational', 'traditional', 'reference', 'colonizer', 'plotted']
single_vocab = [stemmer.stem(plural) for plural in vocab]
single_vocab = np.unique(single_vocab)

In [ ]:
print(len(vocab))
print(len(single_vocab))

91491
86225


In [ ]:
print_long_list(single_vocab, limit = (0,20000))

$	$100	$133	$223	$226	$297	$320	$339	$402	$553	

$birt_hom	$date	$date$	$id	$id$	$odadriv	$revis	$revision$	8859_1	_	

_1partid	_1partqueryresultid	_25_liter	_2partid	_2partqueryresultid	___	__activeid	__authent	__bean	__beforeendtag	

__bookmark	__checkdocumentexist	__checkhtmlformat	__createprogressbar	__docnam	__doget	__dopost	__equalscalc	__execut	__executeact	

__format	__getcontenttypebyid	__getcontext	__getreportdocu	__handleappear	__handlebodi	__handlebodystyl	__handlecascadinglistbox	__handlecheckbox	__handlecommonlistbox	

__handledivappear	__handleexcept	__handlegeneraldefinit	__handlehead	__handlehidden	__handleifram	__handleiframedefinit	__handlelistbox	__handlemultilistbox	__handlenonsoapexcept	

__handleoutputreport	__handleparamet	__handleradiobutton	__handlescript	__handlestyl	__handletextbox	__hashcodecalc	__init	__initparamet	__iscomplet	

__local	__node	__page	__pagenumb	__process	__processwithdiv	__processwithifram	__renderdocu	__renderreport	__report	

__reportdes

$V$: vocab

$r_i = [w_{t,r_i}|t ∈ V ]$

$s_i = [w_{t,s_i}|t ∈ V ]$

all_wr = [**r1**, **r2**, **r3**,...]

all_ws = [**s1**, **s2**, **s3**,...]

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
all_ws = tf_idf(single_vocab, [get_content(f) for f in all_src_path])
all_wr = tf_idf(single_vocab, get_content_bug_rp(bug_rp_df))
feature_1 = linear_kernel(all_wr, all_ws, dense_output=False)
del all_ws
del all_wr

## Feature 2, 3, 4, 5



### Feature 2: Collaborative filtering
Given a bug report r and a source code file s, let
$br(r, s)$ be the set of bug reports for which file s was fixed before r was reported.

The feature computes the textual similarity between the text
of the current bug report r and the summaries of all the bug
reports in $br(r, s)$.
$$ϕ_2(r, s) = sim(r, br(r, s))$$

In [ ]:
#compute feature 2 for all s,r


In [ ]:
def compute_feature_2(bug_index, src_index, bug_content, bbr_df,
                      all_src_path = all_src_path):
    br_content = " ".join(get_content_bug_rp(bbr_df))
    if not br_content.strip():
        return 0
    wbr = tf_idf(single_vocab, [br_content,bug_content[bug_index]])
    # wr = tf_idf(single_vocab, [bug_content[bug_index]])
    return float(linear_kernel(wbr[0].reshape(1,-1), wbr[1].reshape(1,-1)).squeeze())
def compute_feature_2(bug_indexes, src_indexes):
    for x,y in enumerate(zip(bug_indexes, src_indexes)):
        br_content

In [ ]:
# # test feature 2
# a = np.array([[1,2,3],[3,5,7]])
# # print(a*b.T)
# float(linear_kernel(a[0].reshape(1,-1),a[1].reshape(1,-1)).squeeze())

x = 12
y = 2532
bbr = get_before_bug_df(x,y)
bug_content = get_content_bug_rp(bug_df)
compute_feature_2(12,2532,bug_content, bbr)

0.08036748660008511

### Feature 3: Class Name Similarity
$$ϕ_3(r, s) = \begin{cases}
|s.class| & \text{if } s.class ∈ r \\
0 & \text{otherwise}
\end{cases}$$
$s.class$ denote the name of the main class implemented
in source file $s$, and $|s.class|$ the name length.

In [ ]:
# Feature 3
def get_class_name(f):
    for i in range(len(f)-1,-1,-1):
        if f[i] == "/":
            return f[i+1:len(f)-5]

def compute_feature_3(bug_index, src_index, bug_content, 
                      all_src_path = all_src_path):
    # for i,(x,y) in enumerate(zip(bug_indexes, src_indexes)):
    class_name = get_class_name(all_src_path[src_index])
    return len(class_name) if class_name in bug_content[bug_index] else 0

In [ ]:
# test feature 3
i = 482
print(all_src_path[i])
print(get_class_name(all_src_path[i]))

/content/unzip_source/birt-20140211-1400/UI/org.eclipse.birt.report.designer.ui.editors/src/org/eclipse/birt/report/designer/internal/ui/views/property/GroupPropertyHandleProvider.java
GroupPropertyHandleProvider


### Feature 4: Bug-Fixing Recency
$$ϕ_4(r, s) = (r.month − last(r, s).month + 1)^{−1}$$

$last(r, s) ∈ br(r, s)$ be the most recent previously fixed bug.

If s was last fixed in the same month that r was created, $ϕ_4(r, s)$ is 1. If s was last fixed one month before r was created, $ϕ_4(r, s)$ is 0.5. If $br(r,s) \in \varnothing$, $ϕ_4(r, s)$ is 0

In [ ]:
# Feature 4
#extract month
def get_timestamp(bug_id,bug_df=bug_rp_df):
    return pd.to_datetime(bug_df['report_time'])[bug_id]

def compute_feature_4(bug_index, src_index, bbr_df):
    if bbr_df.empty:
        return 0
    last_time = get_timestamp(bbr_df['report_timestamp'].idxmax(), bug_df=bbr_df)
    bug_time = get_timestamp(bug_index)
    return (int((bug_time-last_time)/np.timedelta64(1, 'M')) + 1)**-1

In [ ]:
# x = 175
# y = 5419
# bbr = get_before_bug_df(x,y)
# print(bug_rp_df.loc[x])
# bbr
# compute_feature_4(x,y,bbr)

### Feature 5: Bug-Fixing Frequency
$$ϕ6(r, s) = |br(r, s)|$$

In [ ]:
def get_before_bug_df(bug_index, src_index, df = bug_rp_df, 
                      all_src_path = all_src_path):
    """Get before bug content
    input:  bug_index taken from bug report dataframe, 
            src path is relative path"""
    report_time = bug_rp_df['report_timestamp'].loc[bug_index]
    src_path = to_relative_path(all_src_path[src_index]) # relative path
    return df.loc[(df.files.str.contains(src_path)) & 
           (df.index != bug_index) & 
           (df['report_timestamp'] < report_time)]

# def compute_features(bug_indexes, src_indexes, bug_df = bug_rp_df):
#     assert len(bug_indexes) == len(src_indexes)

    # feature_2 = np.zeros(len(bug_indexes))
    # feature_3 = np.zeros(len(bug_indexes))
    # feature_4 = np.zeros(len(bug_indexes))
    # feature_5 = np.zeros(len(bug_indexes))
    # for i,(x,y) in enumerate(zip(bug_indexes, src_indexes)):
def compute_features(x,y, bug_content):
    bbr_df = get_before_bug_df(x,y)
    print(x,y)
    try:
        feature_2 =  compute_feature_2(x, y, bug_content, bbr_df)

        feature_3 = compute_feature_3(x, y, bug_content)
        
        feature_4 = compute_feature_4(x, y, bbr_df)

        feature_5 = len(bbr_df)
    except Exception as e:
        print(f"Error on bug_id {x}, source_id {y}")
        print(e)
        traceback.print_exc()

    return feature_2, feature_3, feature_4, feature_5
    # return feature_3, feature_4, feature_5

## Create feature set
- Each bug take all contaminated src and mark them label 1
- Concat those with another 300 bug-free src with lowest tf-idf score

In [ ]:
def create_feature_set(bug_df = bug_rp_df, all_src_path = all_src_path, 
                       feature_1 = feature_1, src_dir = src_dir):

    assert len(bug_rp_df) == feature_1.shape[0]
    assert len(all_src_path) == feature_1.shape[1]

bug_df = bug_rp_df
feature_df = pd.DataFrame(columns=['bug','src','feature_1','label'])
                                # 'feature_3','feature_4','feature_5'])
# bug_src_indexes = []

for i in tqdm(range(len(bug_df))):
    contaminated_src = bug_df['files'].loc[i].split() #relative path
    contaminated_index = [all_src_path.index(to_full_path(f)) for f in contaminated_src]

    not_contaminated_index = [k for k in range(len(all_src_path)) \
                                if k not in contaminated_index]
    negative_300 = sorted(not_contaminated_index, key=lambda y:feature_1[i][y])[:300]
    new_src_index = contaminated_index + negative_300
    labels = [1]*len(contaminated_index) + [0]*len(negative_300)

    # bug_src_indexes.extend([(i,s) for s in new_src_index])
    # bug_indexes, src_indexes = list(zip(*bug_src_indexes))
    feature_df = feature_df.append(pd.DataFrame({'bug': [i]*len(new_src_index),
                        'src': new_src_index,
                        'feature_1': [feature_1[i][y] for y in new_src_index],
                        'label': labels,
                        }),ignore_index=True)
feature_df.head()

    # feature_2, feature_3, feature_4, feature_5 = compute_features(bug_indexes, src_indexes)
                        # 'feature_2': feature_2,
                        # 'feature_3': feature_3,
                        # 'feature_4': feature_4,
                        # 'feature_5': feature_5,
    # return feature_df

100%|██████████| 4178/4178 [03:37<00:00, 19.23it/s]


,bug,src,feature_1,label
0,0,3740,0.10252,1
1,0,196,0.00000,0
2,0,1990,0.00000,0
3,0,1991,0.00000,0
4,0,1993,0.00000,0


In [ ]:
bug_content = get_content_bug_rp(bug_df)
feature_2345_df = feature_df.apply(lambda x: compute_features(x.bug,  x.src,  bug_content),
                                                                axis = 1, result_type='expand', )

0 3740
0 196
0 1990
0 1991
0 1993
0 2000
0 2001
0 2002
0 2003
0 2004
0 2007
0 3364
0 3365
0 3366
0 3597
0 4053
0 4054
0 4055
0 4056
0 4057
0 4058
0 4059
0 4060
0 4061
0 4062
0 4063
0 4064
0 4065
0 4066
0 4067
0 5253
0 5257
0 5263
0 5343
0 5415
0 5416
0 5417
0 5436
0 5497
0 5501
0 5523
0 5539
0 5542
0 5636
0 5638
0 5656
0 5658
0 5682
0 5771
0 5810
0 5921
0 5928
0 6159
0 6289
0 6425
0 6449
0 6572
0 6613
0 6652
0 6695
0 6775
0 6796
0 7002
0 8696
0 8697
0 8703
0 8717
0 8741
0 8750
0 8752
0 8756
0 8758
0 9139
0 9187
0 9318
0 6150
0 6815
0 7972
0 5384
0 8462
0 6741
0 8775
0 1973
0 8167
0 3385
0 2619
0 9011
0 2285
0 6822
0 3384
0 6850
0 5635
0 3645
0 2589
0 6852
0 6876
0 5624
0 9395
0 3371
0 9409
0 8523
0 6855
0 3932
0 6835
0 6831
0 6589
0 3643
0 6448
0 6837
0 6851
0 6810
0 3023
0 4606
0 8543
0 6823
0 6811
0 8751
0 6586
0 1904
0 6873
0 5622
0 3070
0 9404
0 7154
0 8749
0 252
0 9045
0 1310
0 9034
0 9044
0 2006
0 6807
0 5487
0 8722
0 7313
0 1600
0 8723
0 5670
0 6868
0 7693
0 6864
0 8811
0 6778
0

KeyboardInterrupt: ignored

In [ ]:
# without_2_df = pd.concat([feature_2345_df.rename(columns={0:'feature_3',1:'feature_4',2:'feature_5'}),feature_df],axis=1)
# without_2_df

In [ ]:
without_2_df.to_csv("without_2.csv")

In [ ]:
has_2 = without_2_df.apply(lambda x: compute_feature_2(x.bug,  x.src,  bug_content),
                                                                axis = 1, result_type='expand', )